## Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
import numpy as np
import pandas as pd
import csv
import time
import torch
import sklearn

from sklearn.metrics import classification_report
from torch import nn
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from skimage import io, transform
from torchvision import datasets, models, transforms

from utils import label_map_util
from utils import visualization_utils as vis_util

## Car detection stuff...

In [2]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = \
    'http://download.tensorflow.org/models/object_detection/'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [3]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map,
        max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


#### This method counts image num in folder with given ID

In [4]:
def get_max_num(id):
    try:
        images = os.listdir("/Users/andghuladze/Documents/Vision/Myauto_data/Car_Images/Car_Images/{}".format(id))
    except:
        return 0
    return len(images)

#### This method reads image from path and returns tensor image

In [5]:
def get_tensor_image(path):
    path = "/Users/andghuladze/Documents/Vision/Myauto_data/Car_Images/Car_Images/{}".format(path)
    image = transform.resize(io.imread(path), (224, 224))/255
    image = image[:, :, :3]

    sample = torch.Tensor(np.einsum('ijk->kij', image))

    return sample

#### This method returns car images in tensor on given ID.

In [6]:
def object_detection_function(id):
    total_passed_vehicle = 0
    speed = 'waiting...'
    direction = 'waiting...'
    size = 'waiting...'
    color = 'waiting...'
    with detection_graph.as_default():
        with tf.compat.v1.Session(graph=detection_graph) as sess:
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            car_data = []
            img_number = get_max_num(id)
            for i in range(1, img_number + 1):
                input_frame = cv2.imread("/Users/andghuladze/Documents/Vision/Myauto_data/Car_Images/Car_Images/{}/{}.jpg".format(id, i))

                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(input_frame, axis=0)

                # Actual detection.
                (boxes, scores, classes, num) = \
                    sess.run([detection_boxes, detection_scores,
                             detection_classes, num_detections],
                             feed_dict={image_tensor: image_np_expanded})

                # Visualization of the results of a detection.
                vis_util.visualize_boxes_and_labels_on_image_array(
                    0,
                    input_frame,
                    boxes[0],
                    classes[0].astype(np.int32),
                    scores[0],
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=4
                    )

                for j in range(len(classes[0])):
                    if classes[0][j] == 3 or classes[0][j] == 8:
                        if scores[0][j] > 0.5:
                            car_data.append(get_tensor_image(str(id) + "/" + str(i) + ".jpg"))
                            break
                            
            return car_data

## Model initialization. Using Resnet 18.

In [7]:
class model_init(nn.Module):
    def __init__(self):
        super(model_init, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(models.resnet18(pretrained=True))
        self.layers.append(nn.Linear(1000, 512)) 
        self.layers.append(nn.Dropout(0.1))
        self.layers.append(nn.Linear(512, 64))
        self.layers.append(nn.Dropout(0.1))
        self.layers.append(nn.Linear(64, 3))
        self.layers.append(nn.Softmax())
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

#### Model initialization

In [154]:
model = model_init()
model.load_state_dict(torch.load('models/doors_model.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

#### Read csv, remove nulls from doors, shuffle and take proportions as firstly given data.

In [193]:
cars_csv = pd.read_csv('/Users/andghuladze/Documents/Vision/Myauto_data/car_data.csv').dropna(subset=["Doors"]).sample(frac = 1)
four_five = select_column(cars_csv, "Doors", "4/5")[:900]
two_three = select_column(cars_csv, "Doors", "2/3")[:80]
more_five = select_column(cars_csv, "Doors", ">5")[:20]
cars_csv = pd.concat([two_three, four_five, more_five]).sample(frac = 1)

##### Pre defined result categories, because i trained model as given below.

In [194]:
result_categories = ['4/5', '2/3', '>5']

## Classification Report Generation

In [195]:
original_result = []
model_result = []
start = time.time()
for i in range(len(cars_csv)):
    if i % 100 == 0:
        print("TIME ELAPSED: " + str(time.time() - start))
    car_data = object_detection_function(cars_csv.iloc[i].ID)
    try:
        result = model(torch.stack(car_data))
        original_result.append(result_categories.index(cars_csv.iloc[i].Doors))
        winner = 0
        highest = result[:,0].max()
        tmp1 = result[:,1].max()
        tmp2 = result[:,2].max()
        if tmp1 >= highest:
            highest = tmp1
            winner = 1
        if tmp2 >= highest:
            highest = tmp2
            winner = 2
        model_result.append(winner)
    except:
        pass

print("TIME ELAPSED: " + str(time.time() - start))
print(classification_report(original_result, model_result))

TIME ELAPSED: 0.00032401084899902344


/Users/andghuladze/Python/venv_3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


TIME ELAPSED: 516.0010058879852
TIME ELAPSED: 1030.4326810836792
TIME ELAPSED: 1569.4773478507996
TIME ELAPSED: 2107.09237575531
TIME ELAPSED: 2684.6652598381042
TIME ELAPSED: 3251.7176299095154
TIME ELAPSED: 3820.311073064804
TIME ELAPSED: 4356.294437885284
TIME ELAPSED: 4914.247908830643
TIME ELAPSED: 5464.442623138428
              precision    recall  f1-score   support

           0       0.96      0.94      0.95       843
           1       0.15      0.22      0.18        73
           2       0.17      0.05      0.08        19

    accuracy                           0.86       935
   macro avg       0.43      0.40      0.40       935
weighted avg       0.88      0.87      0.87       935



### To check model by giving id's use this method

In [201]:
def check_model(car_ids):
    model_results = []
    for i in range(len(car_ids)):
        car_imgs = object_detection_function(car_ids[i])
        try:
            result = model(torch.stack(car_imgs))
            winner = 0
            highest = result[:,0].max()
            tmp1 = result[:,1].max()
            tmp2 = result[:,2].max()
            if tmp1 >= highest:
                highest = tmp1
                winner = 1
            if tmp2 >= highest:
                highest = tmp2
                winner = 2
            model_results.append(result_categories[winner])
        except:
            pass
    
    return model_results

#### Check model usage

In [202]:
check_model([45681656, 45647533, 45658197, 45770541])

/Users/andghuladze/Python/venv_3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


['4/5', '4/5', '4/5', '4/5']

# Doors evaluation Finished!